# Recurrent switching Linear Dynamical Systems

## Switching linear dynamical systems

At each time $t$, our model has a *discrete latent state* $z_t$ that follows Markovian dynamics with a given Markov transition matrix $\{\pi_k\}_{k=1}^K$:
$$
z_t\in \{1,2,\dots,K\} \\

z_{t+1} | z_t, \{\pi_k\}_{k=1}^K \sim \pi_{z_t}
$$

where $\pi_{z_t}$ is the probability distribution for the next state, i.e. the $z_t$ th row of $\{\pi_k\}_{k=1}^K$.

The discrete state $z_t$ decides which parameters to use for the dynamics of the continuous variables. 
In particular, we have another *latent* but *continuous state* $x_t$ of $M$ variables that follows the conditionally linear dynamic:

_??? A_t? or A_t+1_?

$$
x_{t+1} = A_{z_{t+1}} x_t + b_{z_{t+1}} +v_t \\
v_t \overset{\text{iid}}{\sim} \mathcal{N}(0,Q_{z_{t+1}})
$$

With covariance matrix $Q_{z_{t+1}}\in \mathbb{R}^{M\times M}$, coefficients matrix $A_{z_{t+1}}\in \mathbb{R}^{M\times M}$, and vector $b_{z_{t+1}}\in \mathbb{R}^{M}$ all depending on the state $z$.

And finally, the $N$ variables of the observed time series $y_t$ are assumed to be dependent on the latent variables via a linear relation:

$$
y_{t} = C_{z_t} x_t + d_{z_{t}} +w_t \\
w_t \overset{\text{iid}}{\sim} \mathcal{N}(0,S_{z_{t}})
$$

again, with covariance matrix $S_{z_t}\in \mathbb{R}^{N\times N}$, coefficients matrix $C_{z_{t+1}}\in \mathbb{R}^{N\times M}$, and vector $d_{z_{t+1}}\in \mathbb{R}^{N}$.

In total, we have

$$
\theta = \{ ( \pi_k,A_k,Q_k,b_k,C_k,S_k,d_k ) \}_{k=1}^K
$$

parameters.


## Recurrent Switching State Space Models

But in this model, there is no dependency on the state space; the next state probability $P(z_{t+1})$ is a Markovian *open loop* (???), i.e. it only depends on the current state $z_{t}$. But we want a dependency on the state space as well, and to capture that, we let the transition depend on additional covariates, in particular the latent variable state $x_t$:

$$
z_{t+1} | z_t,x_t,{R_k,r_k} \sim \pi_{SB} (\nu_{t+1}) \\
v_{t+1} = R_{z_t}x_t + r_{z_t}
$$

where $\pi_{SB}$ is a stick breaking logistic regression.

Unlike the standard multiclass logistic regression, which uses the softmax link function (the logistic function), here we use a _stick breaking link function_

$$
\pi_{SB} : \mathbb{R}^{K-1} \to [0,1]^{K}
$$

which maps a real vector ($\nu$) to a normalized probability vector via the stick breaking process:

$$
\pi_{SB}(\nu) = (\pi_{SB}^{(1)}(\nu),\dots,\pi_{SB}^{(K)}(\nu)) \\
\pi_{SB}^{(k)}(\nu) = \sigma(\nu_k) \prod_{j<k} (1-\sigma(\nu_j))=\sigma(\nu_k) \prod_{j<k} \sigma(\nu_j)
$$